In [1]:
#requirements.txt

In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.2 MB/s eta 0:00:00


In [3]:
! pip install datasets transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=12276ca1624f9562fd20450864a33f7effe0358b711447fd0183675558bb5c19
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
from transformers import AutoModelForTokenClassification

systemA = AutoModelForTokenClassification.from_pretrained("emilyblah/distilbert-base-uncased-finetuned-ner")
systemB = AutoModelForTokenClassification.from_pretrained("emilyblah/distilbert-base-uncased-finetuned-5tagOnly-ner")

config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

In [5]:
import numpy as np
import pandas as pd
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer, AdamW
import torch
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.notebook import tqdm
import transformers
import random

In [6]:
dataset = load_dataset("Babelscape/multinerd")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
# generate a sampled test dataset for evaluation due to limited resources, to use full size test data, simply use dataset['test'] in future steps.
test_data_list = list(dataset['test'])

random.shuffle(test_data_list)

sample_size = 0.1

small_test_size = int(sample_size * len(test_data_list))

small_test_list = test_data_list[:small_test_size]

sampled_test = dataset['test'].from_pandas(pd.DataFrame(small_test_list))

In [8]:
# generate the English only and 5 NER tag only test data using the code below
class dataset_preprocessor:
    def __init__(self, data):
        self.data = data

    def extract_lang(self,language):
        self.language = language
        self.extracted_lang_data = []
        for l in self.language:
            for i, e in enumerate(self.data['lang']):
                self.lang_dataset= {'tokens':[], 'ner_tags':[], 'lang':[]}
                if e == l:
                    self.lang_dataset['tokens'] = self.data[i]['tokens']
                    self.lang_dataset['ner_tags']= self.data[i]['ner_tags']
                    self.lang_dataset['lang'] = e
                    self.extracted_lang_data.append(self.lang_dataset)
        return self.extracted_lang_data

    def extract_tags(self, tags, tags_index_dict):
        self.tags = tags #list of wanted tags
        self.tags_index_dict = tags_index_dict
        self.extracted_tag_data = []
        self.indices = []

        for t in self.tags:
            if t == 'O':
                self.indices.append(self.tags_index_dict[t])
            else:
                self.indices.append(self.tags_index_dict['B-' + t])
                self.indices.append(self.tags_index_dict['I-' + t])
        for i,e in enumerate(self.data['ner_tags']):
            self.tags_dataset= {'tokens':[], 'ner_tags':[], 'lang':[]}
            self.new_tags = []
            for each_tag in e:
                if each_tag in self.indices:
                    self.new_tags.append(each_tag)
                elif each_tag not in self.indices:
                    self.new_tags.append(0)
            self.tags_dataset['tokens'] = self.data[i]['tokens']
            self.tags_dataset['ner_tags']= self.new_tags
            self.tags_dataset['lang']= self.data[i]['lang']
            self.extracted_tag_data.append(self.tags_dataset)
        return self.extracted_tag_data

In [9]:
lang_preprocess_test= dataset_preprocessor(dataset['test'])

lang_wanted = ['en']

en_test = lang_preprocess_test.extract_lang(lang_wanted)

In [10]:
tag_preprocess_test = dataset_preprocessor(sampled_test)

wanted_tags =  ['PER','ORG', 'LOC', 'DIS', 'ANIM']
tag_index = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }

tag_test = tag_preprocess_test.extract_tags(wanted_tags, tag_index)

In [11]:
en_test_set = datasets.Dataset.from_pandas(pd.DataFrame(data=en_test))

tag_test_set = datasets.Dataset.from_pandas(pd.DataFrame(data=tag_test))

In [12]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

label_all_tokens = True

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. the label is set to -100 so they are ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, set the label to either the current label or -100, depending on the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
task = 'ner'

tokenized_sampled_og_test_set = sampled_test.map(tokenize_and_align_labels, batched=True) #33598 test data, in all language and all tags
tokenized_en_test_set = en_test_set.map(tokenize_and_align_labels, batched=True) #32908 test data, only English
tokenized_tag_test_set = tag_test_set.map(tokenize_and_align_labels, batched=True) #33598 test data, only 5 wanted ner_tags

Map:   0%|          | 0/33598 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

Map:   0%|          | 0/33598 [00:00<?, ? examples/s]

In [15]:
label_list = list(tag_index.keys())

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [17]:
metric = load_metric("seqeval")

<ipython-input-17-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [18]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

set up systemA for testing

In [19]:
systemA_args = TrainingArguments(
    "emilyblah/distilbert-base-uncased-finetuned-ner",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=5000,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [20]:
systemA_trainer = Trainer(
    systemA,
    systemA_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
systemA_predictions, systemA_labels, _ = systemA_trainer.predict(tokenized_sampled_og_test_set)
systemA_predictions = np.argmax(systemA_predictions, axis=2)

# Remove ignored index (special tokens)
systemA_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemA_predictions, systemA_labels)
]
systemA_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemA_predictions, systemA_labels)
]

systemA_results = metric.compute(predictions=systemA_true_predictions, references=systemA_true_labels)
systemA_results

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'ANIM': {'precision': 0.3447572419420645,
  'recall': 0.10844455852156057,
  'f1': 0.16499072537342577,
  'number': 7792},
 'BIO': {'precision': 0.04,
  'recall': 0.029411764705882353,
  'f1': 0.03389830508474576,
  'number': 34},
 'CEL': {'precision': 0.038527397260273974,
  'recall': 0.06241331484049931,
  'f1': 0.047644256220222336,
  'number': 721},
 'DIS': {'precision': 0.49480249480249483,
  'recall': 0.11021069692058347,
  'f1': 0.18026888846809316,
  'number': 4319},
 'EVE': {'precision': 0.4054878048780488,
  'recall': 0.04944237918215613,
  'f1': 0.08813783962889331,
  'number': 2690},
 'FOOD': {'precision': 0.1377245508982036,
  'recall': 0.08789808917197452,
  'f1': 0.10730948678071538,
  'number': 2355},
 'INST': {'precision': 0.37037037037037035,
  'recall': 0.04424778761061947,
  'f1': 0.07905138339920947,
  'number': 226},
 'LOC': {'precision': 0.6504188418113391,
  'recall': 0.260249684251433,
  'f1': 0.37175172605211115,
  'number': 41172},
 'MEDIA': {'precision': 0.

In [22]:
systemA_predictions, systemA_labels, _ = systemA_trainer.predict(tokenized_en_test_set)
systemA_predictions = np.argmax(systemA_predictions, axis=2)

# Remove ignored index (special tokens)
systemA_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemA_predictions, systemA_labels)
]
systemA_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemA_predictions, systemA_labels)
]

systemA_results = metric.compute(predictions=systemA_true_predictions, references=systemA_true_labels)
systemA_results

{'ANIM': {'precision': 0.731415643180349,
  'recall': 0.7697278911564626,
  'f1': 0.7500828637719588,
  'number': 5880},
 'BIO': {'precision': 0.5526315789473685,
  'recall': 0.7,
  'f1': 0.6176470588235295,
  'number': 60},
 'CEL': {'precision': 0.6461538461538462,
  'recall': 0.8076923076923077,
  'f1': 0.717948717948718,
  'number': 104},
 'DIS': {'precision': 0.7639186295503212,
  'recall': 0.8121798520204895,
  'f1': 0.7873103448275861,
  'number': 3514},
 'EVE': {'precision': 0.9191011235955057,
  'recall': 0.9423963133640553,
  'f1': 0.9306029579067121,
  'number': 868},
 'FOOD': {'precision': 0.644468313641246,
  'recall': 0.6263048016701461,
  'f1': 0.6352567496029645,
  'number': 1916},
 'INST': {'precision': 0.5925925925925926,
  'recall': 0.8,
  'f1': 0.6808510638297872,
  'number': 40},
 'LOC': {'precision': 0.9744058500914077,
  'recall': 0.9738124238733252,
  'f1': 0.9741090466037161,
  'number': 36124},
 'MEDIA': {'precision': 0.9201331114808652,
  'recall': 0.958405545

set up systemB for testing

In [23]:
systemB_args = TrainingArguments(
    "emilyblah/distilbert-base-uncased-finetuned-5tagOnly-ner",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=5000,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [24]:
systemB_trainer = Trainer(
    systemB,
    systemB_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
systemB_predictions, systemB_labels, _ = systemB_trainer.predict(tokenized_sampled_og_test_set)
systemB_predictions = np.argmax(systemB_predictions, axis=2)

# Remove ignored index (special tokens)
systemB_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]
systemB_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]

systemB_results = metric.compute(predictions=systemB_true_predictions, references=systemB_true_labels)
systemB_results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ANIM': {'precision': 0.6195681151692882,
  'recall': 0.5965092402464066,
  'f1': 0.6078200601543089,
  'number': 7792},
 'BIO': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 34},
 'CEL': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 721},
 'DIS': {'precision': 0.6278118609406953,
  'recall': 0.5686501504978004,
  'f1': 0.5967683149070586,
  'number': 4319},
 'EVE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2690},
 'FOOD': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2355},
 'INST': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 226},
 'LOC': {'precision': 0.8724456818972803,
  'recall': 0.8523996891091032,
  'f1': 0.8623061991695128,
  'number': 41172},
 'MEDIA': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 4451},
 'MYTH': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 651},
 'ORG': {'precision': 0.7885441527446301,
  'recall': 0.7573349633251834,
  'f1': 0.7726245225660613,
  'number': 6544},
 'PER': {'precision': 0.

In [25]:
systemB_predictions, systemB_labels, _ = systemB_trainer.predict(tokenized_tag_test_set)

# Remove ignored index (special tokens)
systemB_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]
systemB_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]

systemB_results = metric.compute(predictions=systemB_true_predictions, references=systemB_true_labels)
systemB_results

{'ANIM': {'precision': 0.6195681151692882,
  'recall': 0.5965092402464066,
  'f1': 0.6078200601543089,
  'number': 7792},
 'DIS': {'precision': 0.6278118609406953,
  'recall': 0.5686501504978004,
  'f1': 0.5967683149070586,
  'number': 4319},
 'LOC': {'precision': 0.8724456818972803,
  'recall': 0.8523996891091032,
  'f1': 0.8623061991695128,
  'number': 41172},
 'ORG': {'precision': 0.7885441527446301,
  'recall': 0.7573349633251834,
  'f1': 0.7726245225660613,
  'number': 6544},
 'PER': {'precision': 0.8853491801081275,
  'recall': 0.8801581308577267,
  'f1': 0.8827460239675681,
  'number': 22513},
 'overall_precision': 0.8339351978681543,
 'overall_recall': 0.8133349526354141,
 'overall_f1': 0.8235062651402432,
 'overall_accuracy': 0.9719070093440144}